# COGS 108 - Final Project 

# Overview

*Fill in your overview here*

Unlike other business, parks usually are not very popular to be reviewed on Yelp. It's very interesting to see if some areas in San Diego are  more likely to receive park reviews, and why.

This project analyzed Yelp review data, labelled each park with the census tract in San Diego, and compared different factors that may affect the number of total reviews for each census tract. But there are no strong evidence to conclude they have statistically significant relationships.

# Name & GitHub

- Name: Jinmou Li
- GitHub Username: jil732

# Research Question

*Fill in your research question here*

Why do different areas in San Diego have different average number of reviews for review parks and recreation centers on Yelp?

## Background and Prior Work

*Fill in your background and prior work here* 

People are very used to reviewing restruants and other places on Yelp, but what about parks and recreation centers?

Yelp is known for the platform to review businesses (1). Usually when people don't know if a local business is good, such as a resturant, they will refer to the reviews on Yelp. On the other hand, parks are usually operated by the city, and not usually labelled as local business.

"A park is an area of natural, semi-natural or planted space set aside for human enjoyment and recreation or for the protection of wildlife or natural habitats. Urban parks are green spaces set aside for recreation inside towns and cities. National parks and country parks are green spaces used for recreation in the countryside. State parks and provincial parks are administered by sub-national government states and agencies" (2).

"A recreation center is a place for recreational activities usually administered by a municipal government agency." (3)

"A census tract, census area, census district or meshblock[1] is a geographic region defined for the purpose of taking a census.[2] Sometimes these coincide with the limits of cities, towns or other administrative areas[2] and several tracts commonly exist within a county. Census tracts represent the smallest territorial entity for which population data are available in many countries.[3] In the United States, census tracts are subdivided into block groups and census blocks" (4). In this project, we use the census tract as the geography unit for our data.

The number of prior work on what motivates people to leave more reviews on Yelp for San Diego parks and recreation centers is very limited. As a result, I didn't have the luck to find any related work before the deadline of this project. I defintiely hope to invest more time on this part and unfortunately I don't think I can include any here.

For the gerneral number of reviews on Yelp, I found some information online (5). First, businesses on Yelp should encourage the customers to post reviews on Yelp. If there is any park that encourages the customers to do so, it's reasonable to expect them to have more reviews. But I don't have the data on this so it's not included in this project. Second, businesses should try to offer exclusive deals for customers using Yelp. I don't think this makes sense for most parks.

A most similar work to this project is (6). It is related since it's also spatial analysis and focused on clusters of business. For this project, however, we don't necessarily analysze the clustering of parks.

References (include links):
- 1) https://en.wikipedia.org/wiki/Yelp
- 2) https://en.wikipedia.org/wiki/Park
- 3) https://en.wikipedia.org/wiki/Recreation#Recreation_center
- 4) https://en.wikipedia.org/wiki/Census_tract
- 5) https://www.webstaurantstore.com/article/36/how-to-get-more-yelp-reviews.html
- 6) https://opengeospatialdata.springeropen.com/articles/10.1186/s40965-017-0020-9

# Hypothesis


*Fill in your hypotheses here*

The areas with the most popular parks are more likely to have higher average number of reviews on Yelp.

Reason: since we are calculating the average, the parks with the most reviews naturally will affect the average number in each area.

# Dataset(s)

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name:
- Link to the dataset:
- Number of observations:

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

- Dataset Name: Park Locations Data
  - Link to the dataset: https://data.sandiego.gov/datasets/park-locations/
  - Number of observations: 2769
  - Used only to get the names of some San Diego parks and recreation centers

- Dataset Name: Rec Center Locations Data
  - Link to the dataset: https://data.sandiego.gov/datasets/recreation-center-locations/
  - Number of observations: 57
  - Used only to get the names of some San Diego parks and recreation centers

- Dataset Name: Joint Use Parks Data
  - Link to the dataset: https://data.sandiego.gov/datasets/joint-use/
  - Number of observations: 96
  - Used only to get the names of some San Diego parks and recreation centers

- Dataset Name: Yelp Data for San Diego Parks
  - Link to the dataset: https://github.com/COGS108/individual_fa20/blob/master/data/yelp_SD_parks.csv
  - Number of observations: 833

- Dataset Name: California Environmental Health Screening Data
  - Link to the dataset: https://data.ca.gov/dataset/calenviroscreen-2-0
  - Number of observations: 8035

- Dataset Name: 2019 TIGER/Line® Shapefiles: Census Tracts
  - Link to the dataset: https://www.census.gov/cgi-bin/geo/shapefiles/index.php?year=2019&layergroup=Census+Tracts
  - Number of observations: 8057

- Dataset Name: ADDRESS_APN
  - Link to the dataset: https://rdw.sandag.org/Account/gisdtview?dir=Address
  - Number of observations: 1166568
  - Used only to get zip codes of San Diego
  
### How to combine

For the first 3 data sets, they are used only to get a set of names of San Diego parks and recreastion ceters to filter the data of the yelp data.

The screening data and the census tract data are combined with the tract ID, and used to label the yelp data and visualize the distribution of average review counts.

Those 2 data sets are combined with the yelp data using the latitude and longitude.

For the last dataset, its only use is to get San Diego zip codes.

# Setup

In [1]:
## YOUR CODE HERE
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
   
import seaborn as sns
sns.set()
sns.set_context('talk')

import re
from scipy.spatial import cKDTree

from shapely.ops import nearest_points

import patsy
import statsmodels.api as sm
import scipy.stats as stats

# Data Cleaning

Describe your data cleaning steps here.

After converting the strings to lower cases, unnecessary data are dropped, either because those columns are not used in this project or the rows do not belong to San Diego.

In [ ]:
# Read the data

df_parkl = gpd.read_file('../data/parks_datasd.geojson')
df_dict_parkl = pd.read_csv('../data/parks_dictionary_datasd.csv')
df_recl = gpd.read_file('../data/rec_centers_datasd.geojson')
df_joint = gpd.read_file('../data/joint_use_datasd.geojson')

df_ypark = pd.read_csv('../data/yelp_SD_parks.csv', dtype={'phone':str})

df_dict_screen = pd.read_excel('../data/cal-enviro-screen-data-dictionary.xlsx')
df_screen = pd.read_csv('../data/calenviroscreen-final-report.csv')

gdf_census_tracts = gpd.read_file('../data/tl_2019_06_tract/tl_2019_06_tract.shp')
gdf_census_tracts['GEOID'] = gdf_census_tracts['GEOID'].apply(int)

shp_address_apn = gpd.read_file('../data/Address_APN/ADDRESS_APN.shp')

In [ ]:
# Shape of data

print(df_parkl.shape)
print(df_recl.shape)
print(df_joint.shape)
print(df_ypark.shape)
print(df_screen.shape)
print(gdf_census_tracts.shape)
print(shp_address_apn.shape)

## Quick peek for each dataset:

In [ ]:
df_parkl.dtypes

In [ ]:
df_dict_parkl

In [ ]:
df_parkl.head()

In [ ]:
df_recl.dtypes

In [ ]:
df_recl.head()

In [ ]:
df_joint.dtypes

In [ ]:
df_joint.head()

In [ ]:
df_ypark.dtypes

In [ ]:
df_ypark.head()

In [ ]:
# Number of parks
len(df_ypark)

In [ ]:
# Number of parks with less than 50 reviews
len(df_ypark[df_ypark['review_count'] < 50])

In [ ]:
# Number of parks with less than 100 reviews
len(df_ypark[df_ypark['review_count'] < 100])

In [ ]:
# Dictionary of screen data
df_dict_screen

In [ ]:
df_screen.dtypes

In [ ]:
df_screen.head()

In [ ]:
# List the counties in the dataset
df_screen['California County'].unique()

In [ ]:
gdf_census_tracts.dtypes

In [ ]:
gdf_census_tracts.head()

In [ ]:
shp_address_apn.dtypes

In [ ]:
shp_address_apn.head()

## To lower names

In [ ]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

df_parkl['name'] = df_parkl['name'].apply(str.lower)
df_parkl['alias'] = df_parkl['alias'].apply(str.lower)

df_recl['rec_bldg'] = df_recl['rec_bldg'].apply(str.lower)
df_recl['park_name'] = df_recl['park_name'].apply(str.lower)

df_joint['name'] = df_joint['name'].apply(str.lower)
df_joint['facility'] = df_joint['facility'].apply(str.lower)

df_ypark['name'] = df_ypark['name'].apply(str.lower)

shp_address_apn.dropna(inplace=True, subset=['COMMUNITY', 'ADDRZIP'])
shp_address_apn['COMMUNITY'] = shp_address_apn['COMMUNITY'].apply(str.lower)

### Get the set of "all" possiable names of parks and recreations

In [ ]:
set_park_rec = set(df_parkl['name']).union(
    set(df_parkl['alias'])
).union(
    set(df_recl['rec_bldg'])
).union(
    set(df_recl['park_name'])
).union(
    set(df_joint['name'])
).union(
    set(df_joint['facility'])
)

len(set_park_rec)

### Drop the non-park and non-recreation yelp reviews

In [ ]:
tmp_to_drop = df_ypark[~df_ypark.apply(
    lambda x: 
    (x['name'] in set_park_rec) or
    any(keyword in x['name'] for keyword in [
        'park', 'complex', 'recreation', 'preserve', 'trail', 'pools', 'cove', 'national',
        'beach', 'shores', 'garden', 'plaza', 'reserve', 'playground', 'lake', 'mountain',
        'peak', 'canyon', 'rock', 'landing', 'italy', 'island', 'youth', 'boardwalk', 'photography',
        'kayak', 'community', 'zoo', 'seaworld', 'village', 'valley', 'museum', 'center', 'theatre',
        'pool', 'bridge', 'path', 'aquarium', 'utc', 'racetrack', 'falls', 'old globe', 'tour', 'golf',
        'marathon', 'ranch', 'crown point', 'bay of play', 'lighthouse', 'landscape', 'aquaplex',
        'skyride', 'viewpoint', 'field', 'diego bay', 'summit'
    ])
, axis=1)]
tmp_to_drop.sort_values(by=['review_count'], ascending=False).head()

In [ ]:
df_ypark = df_ypark[df_ypark.apply(
    lambda x: 
    (x['name'] in set_park_rec) or
    any(keyword in x['name'] for keyword in [
        'park', 'complex', 'recreation', 'preserve', 'trail', 'pools', 'cove', 'national',
        'beach', 'shores', 'garden', 'plaza', 'reserve', 'playground', 'lake', 'mountain',
        'peak', 'canyon', 'rock', 'landing', 'italy', 'island', 'youth', 'boardwalk', 'photography',
        'kayak', 'community', 'zoo', 'seaworld', 'village', 'valley', 'museum', 'center', 'theatre',
        'pool', 'bridge', 'path', 'aquarium', 'utc', 'racetrack', 'falls', 'old globe', 'tour', 'golf',
        'marathon', 'ranch', 'crown point', 'bay of play', 'lighthouse', 'landscape', 'aquaplex',
        'skyride', 'viewpoint', 'field', 'diego bay', 'summit'
    ]) and (not ('restaurant' in x['name'])),
axis=1)]

len(df_ypark)

In [ ]:
df_ypark.head()

## Drop useless columns

In [ ]:
df_ypark.drop(inplace=True, columns=['phone', 'id', 'url', 'is_closed'])
df_screen.drop(inplace=True, columns=['Click for interactive map', 'Hyperlink'])
gdf_census_tracts.drop(inplace=True, columns=['STATEFP', 'COUNTYFP', 'TRACTCE', 'NAME', 'NAMELSAD', 'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER'])
shp_address_apn = shp_address_apn[['COMMUNITY', 'ADDRZIP']].drop_duplicates()

## Drop useless dataframe

In [ ]:
df_parkl = None
df_recl = None
df_joint = None

## Drop null rows and useless data

In [ ]:
df_ypark.isnull().sum()

In [ ]:
df_ypark.dropna(inplace=True)

In [ ]:
# Only analysze San Diego data
df_screen = df_screen[df_screen['California County'] == 'San Diego'].drop(columns=['California County'])

In [ ]:
df_screen.isnull().sum()

In [ ]:
df_screen.dropna(inplace=True)

In [ ]:
gdf_census_tracts.isnull().sum()

In [ ]:
# Only collect San Diego census tracts
census_tracts_sd = set(df_screen['Census Tract'])

In [ ]:
gdf_census_tracts.isnull().sum()

In [ ]:
gdf_census_tracts = gdf_census_tracts[gdf_census_tracts['GEOID'].isin(census_tracts_sd)]
gdf_census_tracts.plot(figsize=(10,10), edgecolor="purple", facecolor="None")

## Plot the data and furtuher drop unnecessary rows

### Extract latitude and longtitude informaiton

In [ ]:
# Extract latitude and longtitude from Location 1
df_screen_location_pattern = re.compile(r""".+\n\((?P<latitude>[-+]?\d*\.\d+|\d+?), (?P<longitude>[-+]?\d*\.\d+|\d+?)\)""")

def extract_screen_location(s):
    m = df_screen_location_pattern.match(s)

    la = float(m.group('latitude'))
    lo = float(m.group('longitude'))

    return (la, lo)

df_screen = df_screen.join(df_screen['Location 1'].apply(
    lambda x: pd.Series(extract_screen_location(x), index=['latitude', 'longitude']))).drop(columns=['Location 1'])

In [ ]:
# Plot the distribution of the park relative to the census tracts
gdf_ypark = gpd.GeoDataFrame(df_ypark, geometry=gpd.points_from_xy(df_ypark.longitude, df_ypark.latitude))
gdf_screen = gpd.GeoDataFrame(df_screen, geometry=gpd.points_from_xy(df_screen.longitude, df_screen.latitude))

fig_park_screen, ax_park_screen = plt.subplots(figsize = (40,40)) 
gdf_census_tracts.plot(edgecolor="purple", facecolor="None", ax=ax_park_screen)
gdf_screen.plot(edgecolor="Cyan", facecolor="None", ax=ax_park_screen)
gdf_ypark.plot(edgecolor="None", facecolor="Red", ax=ax_park_screen)

ax_park_screen.set_ylabel('latitude')
ax_park_screen.set_xlabel('longitude')
ax_park_screen.set_title('Park distribution reletive to census tracts')

From the graph, we can simply count and see there are about 20 parks that are not in San Diego census tracts.

Let's try to remove them from the data.

In [ ]:
zip_code_sd_partial = set(shp_address_apn['ADDRZIP'].unique())

In [ ]:
tmp_to_drop = df_ypark[~df_ypark.apply(
    lambda x: 
    (x['address'][-5:] in zip_code_sd_partial) or
    ('San Diego' in x['address']) or
    ('La Jolla' in x['address'])
, axis=1)]

print(len(tmp_to_drop))
tmp_to_drop

This is a good-enough approximation for the data points to drop. Let's remove them from our dataframe.

In [ ]:
df_ypark = df_ypark[df_ypark.apply(
    lambda x: 
    (x['address'][-5:] in zip_code_sd_partial) or
    ('San Diego' in x['address']) or
    ('La Jolla' in x['address'])
, axis=1)]

Let's check the cleaned data.

In [ ]:
# Reset the index after dropping
df_ypark.reset_index(inplace=True, drop=True)
df_screen.reset_index(inplace=True, drop=True)
gdf_census_tracts.reset_index(inplace=True, drop=True)
shp_address_apn.reset_index(inplace=True, drop=True)

In [ ]:
# Check the shapes
print(df_ypark.shape)
print(df_screen.shape)
print(gdf_census_tracts.shape)
print(shp_address_apn.shape)

In [ ]:
# Check the columns and types
df_ypark.dtypes

In [ ]:
df_screen.dtypes

In [ ]:
gdf_census_tracts.dtypes

In [ ]:
shp_address_apn.dtypes

In [ ]:
# Plot the distribution of parks
gdf_ypark = gpd.GeoDataFrame(df_ypark, geometry=gpd.points_from_xy(df_ypark.longitude, df_ypark.latitude))
gdf_screen = gpd.GeoDataFrame(df_screen, geometry=gpd.points_from_xy(df_screen.longitude, df_screen.latitude))

fig_park_screen, ax_park_screen = plt.subplots(figsize = (40,40)) 
gdf_census_tracts.plot(edgecolor="purple", facecolor="None", ax=ax_park_screen)
gdf_screen.plot(edgecolor="Cyan", facecolor="None", ax=ax_park_screen)
gdf_ypark.plot(edgecolor="None", facecolor="Red", ax=ax_park_screen)

ax_park_screen.set_ylabel('latitude')
ax_park_screen.set_xlabel('longitude')
ax_park_screen.set_title('Park distribution reletive to census tracts')

# Data Analysis & Results

Include cells that describe the steps in your data analysis.

## EDA

What distributions do your variables take?

* The review_count is skewed, details (with graphs) are shown later

Are there any outliers?

* Yes, and they are also shown later. And the spatial outliers are removed in the last section

Relationship between variables?

* They are seemingly independent and the analysis will try to find any linear relationship
  
## Analysis

What approaches did you use? Why?

* I used cKDTree to find the nearest census tract center for each park, beacuse it's fast and accuracte enough for the purpose of this project
* I used correlation analysis, specifically the Spearman rank-order correlation coefficient, because it does not assume the data to be normally distributed and it gives p-value
* I used OLS regression, because it's efficient and reliable to analyze the linear relationship among multiple variable and it also outputs p-value

What were the results?

* None of the proposed factors that may affect the average number of reviews are proved to be statically significant

What were your interpretation of these findings

* It's very reasonable, considering most people do not care the review counts on Yelps for parks
* Details are included in the discussion

## Data Visualization 

Included in the following cells

## Analysis

In [ ]:
ax_review_distplot = sns.distplot(df_ypark['review_count'], bins=30, kde=False, color='#686868');
sns.despine()
ax_review_distplot.set_title('')
ax_review_distplot.set_ylabel('Frequency')
ax_review_distplot.set_xlabel('review_count');

We can see the review_count for parks are very skewed.

## Calculate the nearest census tract

We first use cKDTree to label each park with the nearest census tract.

Note we don't analyze the exact boundary of the tracts,

In [ ]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

df_ypark = df_ypark.assign(
    Tract=[df_screen.at[idx, 'Census Tract'] for idx in cKDTree(df_screen[['longitude', 'latitude']]).query(df_ypark[['longitude', 'latitude']])[1]]
)

In [ ]:
# Check the value counts
ypark_tract_counts = df_ypark['Tract'].value_counts()

In [ ]:
# Check the distribution
ypark_tract_counts.sort_values()

It seems that only 4 tracts have more than 9 parks, which seems a good distribution for mapping purpose. There is no big clustering.

In [ ]:
# Number of tracts with only 1 park
ypark_tract_counts.where(ypark_tract_counts == 1, 0).sum()

In [ ]:
# Number of tracts with at least 1 park
len(df_ypark['Tract'].unique())

In [ ]:
# Number of tracts with 0 park
len(df_screen['Census Tract'].unique()) - len(df_ypark['Tract'].unique())

In [ ]:
# Calculate the average of review counts for each census tract
df_tract_review = df_ypark.groupby('Tract')['review_count'].mean().to_frame().reset_index()
df_tract_review.head()

In [ ]:
# Use 0 as the average for the tracts without parks
df_tract_review_zero = pd.DataFrame(list(set(df_screen['Census Tract'].unique())-set(df_tract_review['Tract'].unique())), columns=['Tract'])
df_tract_review_zero['review_count'] = float(0)
df_tract_review = pd.concat([df_tract_review, df_tract_review_zero]).sort_values(by=['Tract']).reset_index(drop=True)
df_tract_review.head()

In [ ]:
# Check the distribution and variance
df_tract_review.sort_values(by=['review_count'], ascending=False)[:10]

In [ ]:
df_tract_review.var()

From above, however, we see the average number of reviews have non-minor variance, and the largest one is more than 2 times larger than the second largets value.

We visualize them on the map to have to better idea.

In [ ]:
df_tract_review = df_tract_review.merge(gdf_census_tracts[['GEOID', 'geometry']], left_on='Tract', right_on='GEOID').drop(columns=['GEOID'])
gdf_tract_review = gpd.GeoDataFrame(df_tract_review, geometry='geometry')
gdf_tract_review.head()

In [ ]:
fig_tract_review, ax_tract_review = plt.subplots(figsize = (40,40)) 
gdf_tract_review.boundary.plot(edgecolor="purple", facecolor="None", ax=ax_tract_review);
gdf_tract_review.plot(column='review_count', cmap='Reds', ax=ax_tract_review)
ax_tract_review.set_ylabel('latitude')
ax_tract_review.set_xlabel('longitude')
ax_tract_review.set_title('Choropleth Map for Average Review Count')

From the graph, we can see there are few tracts with very high average review counts.

For other tracts, there is not hugh difference considering there are a limited number of popular parks in San Diego.

As a result, we try to visualize the distribution of the review counts without the most popular ones and see what happens.

In [ ]:
# Find the most popular parks and check their tracts
df_ypark.sort_values(by=['review_count'], ascending=False).head(10)

In [ ]:
# Let's remove the top 4 and see what happens
df_ypark_trimed = df_ypark[~df_ypark['name'].isin(set(['san diego zoo', 'seaworld san diego', 'san diego zoo safari park', 'balboa park']))]

df_tract_review_trimed = df_ypark_trimed.groupby('Tract')['review_count'].mean().to_frame().reset_index()

df_tract_review_zero_trimed = pd.DataFrame(list(set(df_screen['Census Tract'].unique())-set(df_tract_review_trimed['Tract'].unique())), columns=['Tract'])
df_tract_review_zero_trimed['review_count'] = float(0)
df_tract_review_trimed = pd.concat([df_tract_review_trimed, df_tract_review_zero_trimed]).sort_values(by=['Tract']).reset_index(drop=True)

df_tract_review_trimed = df_tract_review_trimed.merge(gdf_census_tracts[['GEOID', 'geometry']], left_on='Tract', right_on='GEOID').drop(columns=['GEOID'])
gdf_tract_review_trimed = gpd.GeoDataFrame(df_tract_review_trimed, geometry='geometry')

fig_tract_review_trimed, ax_tract_review_trimed = plt.subplots(figsize = (40,40)) 
gdf_tract_review_trimed.boundary.plot(edgecolor="purple", facecolor="None", ax=ax_tract_review_trimed);
gdf_tract_review_trimed.plot(column='review_count', cmap='Reds', ax=ax_tract_review_trimed)

ax_tract_review_trimed.set_ylabel('latitude')
ax_tract_review_trimed.set_xlabel('longitude')
ax_tract_review_trimed.set_title('Choropleth Map for Average Review Count without Most Popular Parks')

In [ ]:
df_ypark_trimed.sort_values(by=['review_count'], ascending=False).head(10)

### Analysis on trimmed dataset and map

When removing several outliers, we can see more tracts are having similarlly many review counts.

This is a hint that it's very likely for the popular parks to be the driving factors for review counts.

Let's go back to the complete dataset (not trimmed one)

Next step is to see if this is caused by the most popular parks in San Diego.

Because many tracts do not have any park, and our focus is the park not each tract, let's ignore the tracts with no park.

In [ ]:
# Analyze the tracts with at least one park
df_tract_review_one = df_tract_review[df_tract_review['review_count'] != 0]
gdf_tract_review_one = gpd.GeoDataFrame(df_tract_review_one, geometry='geometry')

fig_tract_review_one, ax_tract_review_one = plt.subplots(figsize = (40,40)) 
gdf_tract_review_one.boundary.plot(edgecolor="purple", facecolor="None", ax=ax_tract_review_one);
gdf_tract_review_one.plot(column='review_count', cmap='Reds', ax=ax_tract_review_one)

ax_tract_review_one.set_ylabel('latitude')
ax_tract_review_one.set_xlabel('longitude')
ax_tract_review_one.set_title('Choropleth Map for Average Review Count with At Least One Park')

This map is simply what we have seen without the census tracts with no parks.

To begin with, let's check the distribution of the reveiw_count (i.e. the popularity of parks)

In [ ]:
ax_tract_distplot = sns.distplot(df_tract_review_one['review_count'], bins=30, kde=False, color='#686868');
sns.despine()
ax_tract_distplot.set_title('')
ax_tract_distplot.set_ylabel('Frequency')
ax_tract_distplot.set_xlabel('review_count');

We can see the data is very skewed.

In [ ]:
ax_tract_distplot = sns.distplot(df_tract_review_one[df_tract_review_one['review_count'] > 500]['review_count'], bins=30, kde=False, color='#686868');
sns.despine()
ax_tract_distplot.set_title('')
ax_tract_distplot.set_ylabel('Frequency')
ax_tract_distplot.set_xlabel('review_count > 500');

We can see there are only 7 parks with review_count > 500.

In [ ]:
ax_tract_distplot = sns.distplot(df_tract_review_one[df_tract_review_one['review_count'] < 500]['review_count'], bins=30, kde=False, color='#686868');
sns.despine()
ax_tract_distplot.set_title('')
ax_tract_distplot.set_ylabel('Frequency')
ax_tract_distplot.set_xlabel('review_count < 500');

This is the distribution for parks with review_count < 500.

It's still very skewed.

In [ ]:
ax_tract_distplot = sns.distplot(df_tract_review_one[df_tract_review_one['review_count'] < 200]['review_count'], bins=30, kde=False, color='#686868');
sns.despine()
ax_tract_distplot.set_title('')
ax_tract_distplot.set_ylabel('Frequency')
ax_tract_distplot.set_xlabel('review_count < 200');

Even for the parks with review_count < 200, it's still highly skewed. A lot of parks do not have more than 50 reviews.

### Analysis on park popularity

From the histrams, we can see for the majority of the data points, the parks do not have more than 50 reviews.

And for the rest of the dataset, only a few parks have more than 100 reviews.

Let's check the parks in the tracts with the highest review_count

In [ ]:
df_tract_review_one.sort_values(by=['review_count'], ascending=False).head()

In [ ]:
df_ypark_popular = df_ypark[df_ypark['Tract'].isin(
    set(df_tract_review_one.sort_values(by=['review_count'], ascending=False).head(10)['Tract'])
)].sort_values(by=['Tract'])
df_ypark_popular.head()

In [ ]:
df_ypark_popular.groupby('Tract')['review_count'].count().sort_values()

We can see for the tracts with most review_count, many only have one or two parks that are extremely popular.

Let's try to visualize the distribution after dropping those tracts.

In [ ]:
set_partial_popular = set(['6073008901','6073017021','6073020710', '6073007301', '6073009805', '6073021800', '6073005100','6073008312'])
df_tract_review_tail = df_tract_review_one[~df_tract_review_one['Tract'].isin(set_partial_popular)]
gdf_tract_review_tail = gpd.GeoDataFrame(df_tract_review_tail, geometry='geometry')

fig_tract_review_tail, ax_tract_review_tail = plt.subplots(figsize = (40,40)) 
gdf_tract_review_tail.boundary.plot(edgecolor="purple", facecolor="None", ax=ax_tract_review_tail);
gdf_tract_review_tail.plot(column='review_count', cmap='Reds', ax=ax_tract_review_tail)
gdf_ypark.plot(edgecolor="None", facecolor="Red", ax=ax_tract_review_tail)

ax_tract_review_tail.set_ylabel('latitude')
ax_tract_review_tail.set_xlabel('longitude')
ax_tract_review_tail.set_title('Choropleth Map for Average Review Count without Most Popular Parks')

From the first sight, we can see the more parks on tract has, it's more likely to have higher number of reviews.

Let's verify this by calculating the correlation between the number of parks and the number of reviews.

For this project, we use an **alpha significance value of 0.05**.

We calculate the correlation and the p-value between the number of the reviews and

* number of parks
* number of parks with more than 100 reviews
* number of parks with more than 200 reviews
* number of parks that is the top 100 popular ones

In [ ]:
set_top_popular_parks = set(df_ypark.sort_values(by='review_count').head(100)['name'])

In [ ]:
df_tract_review = df_tract_review.merge(
    df_ypark.groupby('Tract').count().reset_index()[['Tract', 'name']].rename(columns={'name':'park_count'}),
    on='Tract').merge(
    df_ypark[df_ypark['review_count'] > 100].groupby('Tract').count().reset_index()[['Tract', 'name']].rename(columns={'name':'popular_park_count'}),
    on='Tract').merge(
    df_ypark[df_ypark['review_count'] > 200].groupby('Tract').count().reset_index()[['Tract', 'name']].rename(columns={'name':'very_popular_park_count'}),
    on='Tract').merge(
    df_ypark[df_ypark['name'].isin(set_top_popular_parks)].groupby('Tract').count().reset_index()[['Tract', 'name']].rename(columns={'name':'top_popular_park_count'}),
    on='Tract')
df_tract_review.head()

In [ ]:
stats.spearmanr(df_tract_review['review_count'], df_tract_review['park_count'])

In [ ]:
stats.spearmanr(df_tract_review['review_count'], df_tract_review['popular_park_count'])

In [ ]:
stats.spearmanr(df_tract_review['review_count'], df_tract_review['very_popular_park_count'])

In [ ]:
stats.spearmanr(df_tract_review['review_count'], df_tract_review['top_popular_park_count'])

### Analysis on correlation with number of parks

None of the p-values are small enough to reject the hypothesis.

This means the number of reviews are positively correlated with the number of the parks with more than 100 or 200 reviews, but this is not statically significant.

We try to calculate the correlation between the review_count and other data of each tract and see if there is anything interesting.

In [ ]:
df_tract_merged = df_tract_review.merge(df_screen, left_on='Tract', right_on='Census Tract').drop(columns=['Census Tract'])
df_tract_merged.head()

In [ ]:
df_tract_merged.corr(method='spearman')['review_count'].sort_values(ascending=False).head(15)

In [ ]:
stats.spearmanr(df_tract_merged['review_count'], df_tract_merged['Traffic'])

In [ ]:
stats.spearmanr(df_tract_merged['review_count'], df_tract_merged['Poverty'])

In [ ]:
stats.spearmanr(df_tract_merged['review_count'], df_tract_merged['Solid Waste'])

In [ ]:
stats.spearmanr(df_tract_merged['review_count'], df_tract_merged['Pesticides'])

### Analysis on correlation

None of those are statically significant, even though Traffic and Poverty seems to be more correlated.

In [ ]:
outcome_review, predictors_review = patsy.dmatrices('review_count ~ park_count + popular_park_count + very_popular_park_count + top_popular_park_count', df_tract_merged)
mod_review = sm.OLS(outcome_review, predictors_review)
res_review = mod_review.fit()

In [ ]:
print(res_review.summary())

In [ ]:
outcome_review, predictors_review = patsy.dmatrices('review_count ~ park_count + popular_park_count + very_popular_park_count + top_popular_park_count + Traffic + Poverty + Q("Solid Waste") + Pesticides', df_tract_merged)
mod_review = sm.OLS(outcome_review, predictors_review)
res_review = mod_review.fit()

In [ ]:
print(res_review.summary())

### Analysis on OLS Regression

Based on all the p values, we don't have strong evidence to say any of the factors significantly affect the number of reviews.

This is reasonable. Since none of those factors will directly push any person to leave more reviews on Yelps for parks.

## Result

From all the analysis above, we fail to support our hypothesis with statictically significant evidence. Thus we cannot make conclusion on whether more popular parks means more average reviews on Yelp.

If we slightly loosen the threshold, we can see the number of parks with more than 100 reviews (i.e. the gerneally more popular parks) have positive correleation coefficient with the number of reviews.

# Ethics & Privacy

*Fill in your ethics & privacy discussion here*

This project did not use any personal information. All the data do not direct relate to any individual. For the Yelp data, it may relate to single business, but as the number of reviews do not have any harm on them, and all the inforamtion are public, it does not have high threats towards ethics or privacy.

Did you have permission to use this data / use it for this purpose?

* Yes.

Are there privacy concerns regarding your datasets that you need to deal with, and/or terms of use that you need to comply with?

* No.

Are there potential biases in your dataset(s), in terms of who it composes, and how it was collected, that may be problematic in terms of it allowing for equitable analysis? (For example, does your data exclude particular populations, or is it likely to reflect particular human biases in a way that could be a problem?)

* Yes. Not everyone leaves reviews on Yelp. In fact, most people do not review parks on Yelp. The data collected may have strong bias towards the people who are active on Yelp and who likes to leave reviews.
* But this is not problematic in terms of it allowing for equitable analysis
* Our research goal itself is why for the people who did leave reviews on Yelp behave differently

Are there any other issues related to your topic area, data, and/or analyses that are potentially problematic in terms of data privacy and equitable impact?

* One potential that may be problematic in terms of equitable impact is that many people without enough income do not have access to the internet and do not have the ability to review on Yelp

How will you handle issues you identified?

* This issue is beyond the scope of this project, as we only want to know about the people who did review on Yelp
* If we need to consider those, we need to fundamentally change our reserach question. We cannot include any specific online platform at all
* Even we only consider paper-based census, it still suffers from the possible lack of coverage
* Thus we highlight the platform we conduct the research, i.e. Yelp

# Conclusion & Discussion

*Fill in your discussion information here*

This project analyzed the distribution of the average number of reviews on Yelp for San Diego parks and recreation centers. The primary data used were the Yelp review data and the question is why do different areas in San Diego have different average number of reviews.

My analysis used Spearman rank-order correlation coefficient measure and OLS regression to check the correlation coefficients and the corresponding p-values. No significant relationship can be supported by strong evidence. The number of popular parks do not have statistically significant linear relationships with the average number of reviews. Thus I cannot support the hypothesis.

The limitations are clear: I don't have the data of most factors that directly push more people to leave more Yelp reviews. Also the popular parks are defined with the number of review_count as well. So essentially speaking the analysis is based only on one data. There is no other better souce of the list of popular parks in San Diego used in this project.

Another limitation is the labelling. I used the census tract to label each sub-area of San Diego. But in real life it may be more useful to label using the larger neighborhoods.

This work does not have direct impacts on the society. It might help people to better understand the parks of San Diego, understand what parks or recreation centers are most popular on Yelp, understand that the popular parks do not distribute uniformally across the whole San Diego county (i.e. a few census tracts have the top popular parks). This may help us to not only think about the access to parks of San Diego residents only but also consider the reviews for the parks.